In [150]:
# General packages
import pandas as pd
import numpy as np
import datetime as dt
import pandas_market_calendars as mcal

# Packages related to API
from pandas_datareader import data as pdr
import yfinance as yf
import scipy as sc
yf.pdr_override()

# Packages related to machine learning and for nueral networs
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
import tensorflow.keras.backend as K

# fix random seed for same reproducibility as my results due to stochastic nature of start point
K.clear_session()
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [151]:
def get_trading_days(start_date, end_date):
    nyse = mcal.get_calendar('NYSE')
    schedule = nyse.schedule(start_date=start_date, end_date=end_date)
    trading_days = nyse.valid_days(start_date=schedule.index[0], end_date=schedule.index[-1])
    def makethisdate(date):
        new = date.date()
        new = str(new)
        new = pd.to_datetime(new)
        return new
    trading_days = list(map(makethisdate, trading_days))
    trading_days = pd.DataFrame(trading_days, columns=["dates"])
    trading_days['weekday'] = trading_days['dates'].dt.dayofweek
    trading_days["end_of_week"] = np.where(trading_days['dates'].dt.dayofweek == 4, True,False)
    trading_days["end_of_month"] = np.where(trading_days['dates'].dt.month != trading_days['dates'].shift(-1).dt.month, True,False)
    
    today = dt.date.today()
    csv_path = f"csv_files/trade_dates_{today}.csv"
    trading_days.to_csv(csv_path)
    
    return trading_days


In [3]:
stocks = ['TSLA','XOM']
end1 = dt.date.today()
start1 = end1 - dt.timedelta(days=20*365)
start = start1.strftime('%Y-%m-%d')
end = end1.strftime('%Y-%m-%d')

stockdata = pdr.get_data_yahoo(stocks, start=start, end=end)
stockdata = stockdata['Adj Close']

returns = stockdata.pct_change().dropna(how='all').dropna(axis=1)

[*********************100%***********************]  2 of 2 completed


In [178]:
# Import stock prices into temp table
temp_table = pd.DataFrame()
temp_table['price'] = stockdata[['TSLA']]

# Convert datetimes to dates
dates = temp_table['price'].index.tolist()
def makethisdate(date):
    new = date.date()
    new = str(new)
    new = pd.to_datetime(new)
    return new
dates = list(map(makethisdate, dates))
temp_table.index = dates

# Get list of months
month_df = get_trading_days(start, end)
month_df = pd.DataFrame(month_df)
month_df=month_df[month_df['end_of_month']==True][['dates']]

# filter dates to only include end of month
temp_table = temp_table[temp_table.index.isin(month_df["dates"])]

len(temp_table)


132

In [191]:
sc = MinMaxScaler(feature_range=(0,1))
train_set = temp_table[:-30].values
test_set = temp_table[-30:].values

sc= MinMaxScaler(feature_range=(0,1))
trainig_set_scaled= sc.fit_transform(train_set)

# Create a data structure with 60 timesteps and 1 output
X_train=[] #Independent variables
y_train= [] # Dependent variables 
# I am going to append past 60 days data 
for i in range(10,len(train_set)):
    X_train.append(trainig_set_scaled[i-10:i,0]) # Appending prevois 60 days data not including 60
    y_train.append(trainig_set_scaled[i,0])
    
X_train, y_train= np.array(X_train), np.array(y_train)

# Create a data structure with 10 timesteps and 1 output
X_test=[] #Independent variables
y_test= [] # Dependent variables 
# I am going to append past 60 days data 
for i in range(10,len(test_set)):
    X_test.append(trainig_set_scaled[i-10:i,0]) # Appending prevois 60 days data not including 60
    y_test.append(trainig_set_scaled[i,0])
    
X_test, y_test= np.array(X_test), np.array(y_test)

# LSMT Model needs to be 3- dimensional, so need to rehsape the x_train, y_train
X_train= np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))

# LSMT Model needs to be 3- dimensional, so need to rehsape the x_train, y_train
X_test= np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))


# Importing the Keras libraries and packages
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

#  Initialising the RNN
model= Sequential()

# Adding first LSTM layer and some dropout Dropout regularisation
model.add(LSTM(units=100,return_sequences=True, input_shape=(X_train.shape[1],1)))
model.add(Dropout(rate=0.2))

# Adding second LSTM layer and some dropout Dropout regularisation
model.add(LSTM(units=100,return_sequences=True))
model.add(Dropout(rate=0.2))

# Adding third LSTM layer and some dropout Dropout regularisation
model.add(LSTM(units=100,return_sequences=True))
model.add(Dropout(rate=0.2))

# Adding fourth LSTM layer and some dropout Dropout regularisation
model.add(LSTM(units=100,return_sequences=True))
model.add(Dropout(rate=0.2))

# Adding fifth LSTM layer and some dropout Dropout regularisation
model.add(LSTM(units=100))
model.add(Dropout(rate=0.2))

# Adding the Output Layer
model.add(Dense(units=1))

# Compiling the Model
# Because we're doing regression hence mean_squared_error
model.compile(loss='mean_squared_error', optimizer='adam')

# Fitting the model to the Training set
history=model.fit(X_train,y_train,epochs=100,batch_size=12)


Epoch 1/100
8/8 [==============================] - 6s 22ms/step - loss: 0.0287
Epoch 2/100
8/8 [==============================] - 0s 20ms/step - loss: 0.0248
Epoch 3/100
8/8 [==============================] - 0s 20ms/step - loss: 0.0241
Epoch 4/100
8/8 [==============================] - 0s 21ms/step - loss: 0.0247
Epoch 5/100
8/8 [==============================] - 0s 20ms/step - loss: 0.0187
Epoch 6/100
8/8 [==============================] - 0s 19ms/step - loss: 0.0178
Epoch 7/100
8/8 [==============================] - 0s 20ms/step - loss: 0.0112
Epoch 8/100
8/8 [==============================] - 0s 19ms/step - loss: 0.0140
Epoch 9/100
8/8 [==============================] - 0s 21ms/step - loss: 0.0159
Epoch 10/100
8/8 [==============================] - 0s 20ms/step - loss: 0.0164
Epoch 11/100
8/8 [==============================] - 0s 21ms/step - loss: 0.0162
Epoch 12/100
8/8 [==============================] - 0s 20ms/step - loss: 0.0195
Epoch 13/100
8/8 [==============================]

In [192]:
predicted_stock_price = model.predict(X_test)

1/1 [==============================] - 1s 1s/step


In [197]:
predicted_stock_price2 = sc.inverse_transform(predicted_stock_price)
print(predicted_stock_price)
print(predicted_stock_price2)

[[1070.936 ]
 [1082.4138]
 [1099.3339]
 [1133.9089]
 [1202.4061]
 [1310.2714]
 [1458.4951]
 [1649.2899]
 [1847.588 ]
 [2000.1615]
 [2063.4062]
 [2047.6501]
 [2021.6177]
 [2056.313 ]
 [2119.6982]
 [2217.0686]
 [2352.5583]
 [2497.3582]
 [2575.5366]
 [2580.3008]]
[[175933.78]
 [177819.33]
 [180598.94]
 [186278.88]
 [197531.52]
 [215251.47]
 [239601.47]
 [270945.  ]
 [303521.12]
 [328585.72]
 [338975.5 ]
 [336387.06]
 [332110.5 ]
 [337810.22]
 [348223.06]
 [364218.94]
 [386477.  ]
 [410264.53]
 [423107.6 ]
 [423890.25]]


In [196]:
temp_table[-30:]

,price
2020-11-30,189.199997
2020-12-31,235.223328
2021-01-29,264.510010
2021-02-26,225.166672
2021-03-31,222.643326
2021-04-30,236.479996
2021-05-28,208.406662
2021-06-30,226.566666
2021-07-30,229.066666
2021-08-31,245.240005
